In [1]:
import csv, operator

In [2]:
path = "/home/tiratatp/Repositories/snia_traces/T1/LiveMapsBackEnd/Combined"
trace_file = path + "/disk1_filtered_sheltered.txt"
out_file = path + "/disk1_filtered_sheltered_writeback.txt"
last_block=1953525167

In [3]:
all_sheltered_requests = []
with open(trace_file, "r") as f:
    reader = csv.reader(f, delimiter=" ")
    for line in reader:
        if line[5] == '1':
            all_sheltered_requests.append((int(line[2]) * 512, int(line[3]) * 512))

In [4]:
all_sheltered_requests = sorted(all_sheltered_requests, key=operator.itemgetter(0))

In [11]:
# merge consecutive IOs or overlapping IOs
overlapping = 0
sequential = 0
merged_requests = []
len_all_sheltered_requests = len(all_sheltered_requests)
x = 0
while x < len_all_sheltered_requests:
    req = all_sheltered_requests[x]
    tail = req[0] + req[1]
    y = x + 1
    while y < len_all_sheltered_requests and tail >= all_sheltered_requests[y][0]:
        if tail == all_sheltered_requests[y][0]:
            req = (req[0], req[1] + all_sheltered_requests[y][1])            
            tail += all_sheltered_requests[y][1]    
            sequential += 1
        else:
            overlapping += 1
            tail = max(tail, all_sheltered_requests[y][0] + all_sheltered_requests[y][1])              
        y += 1
    merged_requests.append(req)
    x = y

In [12]:
filtered_requests = []
for r in merged_requests:
    tail = r[0] + r[1]
    if tail < (last_block * 512):
        filtered_requests.append(r)
    elif r[0] < last_block:
        print r[0], r[1]
print "All: %d After merging: %d After filter out OOB: %d" % (len(all_sheltered_requests), len(merged_requests), len(filtered_requests))

All: 1600 After merging: 642 After filter out OOB: 642


In [13]:
with open(out_file, "w") as f:
    f.write("fio version 2 iolog\n")
    f.write("/dev/sdb1 add\n")
    f.write("/dev/sdb1 open\n")    
    for i in filtered_requests:
        f.write("%s %s %d %d\n" % ("/dev/sdb1", "write", i[0], i[1]))
    f.write("/dev/sdb1 close\n")
    

In [14]:
print all_sheltered_requests[:5]
print sum([bcount for _, bcount in all_sheltered_requests])
print sum([bcount for _, bcount in filtered_requests])

[(513802240, 4096), (9007267840, 4096), (10789847040, 4096), (10789851136, 8192), (16347299840, 4096)]
9281024
9281024


In [15]:
print overlapping, sequential

0 958
